In [3]:
from functools import partial
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import numpy as np
from scipy.optimize import curve_fit

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '1JNA2ZuNQuX2TNNFF_vuLC4pIwzz_upBPfuAk1qrCqx4'

In [4]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

sheet = service.spreadsheets()

def load_data():
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range='Data!V2:X').execute()
    values = np.array(result.get('values', []), dtype=float)
    return values[:, 0], values[:, 1], values[:, 2]

def store_data(sol, range, winf):
    values = [winf] + list(sol)
    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=range,
                          body={'values': values}).execute()

In [5]:
x, weight, fat = load_data()

In [6]:
def f(t, w0, Δ0, c4, Δp, τ, winf):
    return winf + (w0 - winf - Δp/2*np.cos(2*np.pi/7*τ) - c4)*2**(-5/7*t) + \
        c4*np.exp(Δ0/c4*t) + Δp/2*np.cos(2*np.pi/7*(t - τ))

def e(x, y, sol, winf):
    return 1 - np.sum((f(x, *sol, winf) - y)**2) / np.sum(np.array(y)**2)

bounds = ((-np.inf, -np.inf, -np.inf, -np.inf, 0), (np.inf, np.inf, np.inf, np.inf, 7))

In [7]:
sol, _ = curve_fit(partial(f, winf=128), x, weight, bounds=bounds)
err = e(x, weight, sol, winf=128)
print("w∞ = 128, w0 = {:g}, Δ0 = {:g}, c4 = {:g}, Δp = {:g}, τ = {:g}, r² = {:.4%}".format(*sol, err))
store_data(sol, 'Data!AB2', winf=128)

w∞ = 128, w0 = 291.016, Δ0 = -0.580208, c4 = 153.844, Δp = 0.824151, τ = 3.52818, r² = 99.9991%


HttpError: <HttpError 403 when requesting https://sheets.googleapis.com/v4/spreadsheets/1JNA2ZuNQuX2TNNFF_vuLC4pIwzz_upBPfuAk1qrCqx4/values/Data%21AB2?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'ACCESS_TOKEN_SCOPE_INSUFFICIENT', 'domain': 'googleapis.com', 'metadata': {'service': 'sheets.googleapis.com', 'method': 'google.apps.sheets.v4.SpreadsheetsService.UpdateValues'}}]">

In [8]:
sol, _ = curve_fit(partial(f, winf=0), x, fat, bounds=bounds)
err = e(x, fat, sol, winf=0)
print("w∞ = 128, w0 = {:g}, Δ0 = {:g}, c4 = {:g}, Δp = {:g}, τ = {:g}, r² = {:.4%}".format(*sol, err))
store_data(sol, 'Data!AC2', winf=0)

w∞ = 128, w0 = 39.1059, Δ0 = -0.098048, c4 = 36.9984, Δp = 0.295114, τ = 1.84496, r² = 99.9912%


HttpError: <HttpError 403 when requesting https://sheets.googleapis.com/v4/spreadsheets/1JNA2ZuNQuX2TNNFF_vuLC4pIwzz_upBPfuAk1qrCqx4/values/Data%21AC2?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'ACCESS_TOKEN_SCOPE_INSUFFICIENT', 'domain': 'googleapis.com', 'metadata': {'service': 'sheets.googleapis.com', 'method': 'google.apps.sheets.v4.SpreadsheetsService.UpdateValues'}}]">